In [1]:
import pandas as pd

# Définir le chemin du fichier
chemin = r"C:\Users\Liam\Desktop\Projet crypto\Data\df_export.csv"

# Importer le DataFrame
df = pd.read_csv(chemin)

# Afficher les premières lignes pour vérifier
print(df.head())


             timestamp     open     high      low    close      volume
0  2021-04-01 00:00:00  1919.37  1937.82  1919.37  1934.49  5054.95614
1  2021-04-01 00:05:00  1935.24  1935.25  1927.86  1927.90  1884.79818
2  2021-04-01 00:10:00  1927.92  1929.63  1923.41  1926.55  2004.92272
3  2021-04-01 00:15:00  1926.55  1928.76  1924.00  1924.80  1469.64577
4  2021-04-01 00:20:00  1924.80  1929.78  1924.28  1928.29  1760.55796


Création d'une base test et train

In [3]:
# Convertir la colonne timestamp en datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])
# Séparer le dataset en deux : avant et après août 2024
train_start = pd.to_datetime("2021-04-01")
train_end = pd.to_datetime("2024-01-01")
test_start = pd.to_datetime("2024-01-01")
test_end = pd.to_datetime("2025-02-26")  # Jusqu'à aujourd'hui

# Séparer les données d'entraînement et de test
train_df = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]
test_df = df[(df["timestamp"] >= test_start) & (df["timestamp"] <= test_end)]

In [9]:
import pandas as pd
import numpy as np

# Fonction pour calculer le RSI
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Fonction pour calculer le MACD
def compute_macd(series, short_window=12, long_window=26, signal_window=9):
    short_ema = series.ewm(span=short_window, min_periods=1).mean()
    long_ema = series.ewm(span=long_window, min_periods=1).mean()
    macd = short_ema - long_ema
    macd_signal = macd.ewm(span=signal_window, min_periods=1).mean()
    return macd, macd_signal

# Fonction pour calculer l'ATR
def compute_atr(train_df, window=14):
    high_low = train_df["high"] - train_df["low"]
    high_close = np.abs(train_df["high"] - train_df["close"].shift())
    low_close = np.abs(train_df["low"] - train_df["close"].shift())
    tr = pd.DataFrame({"high_low": high_low, "high_close": high_close, "low_close": low_close}).max(axis=1)
    return tr.rolling(window=window).mean()

# Fonction pour calculer les autres indicateurs
def compute_indicators(train_df):
    train_df["rsi"] = compute_rsi(train_df["close"], window=14)
    train_df["macd"], train_df["macd_signal"] = compute_macd(train_df["close"])
    train_df["atr"] = compute_atr(train_df)
    train_df["volatility_20d"] = train_df["close"].pct_change().rolling(20).std()
    train_df["roc_5"] = train_df["close"].pct_change(5)
    train_df["roc_10"] = train_df["close"].pct_change(10)
    return train_df


In [10]:
# Appliquer les indicateurs techniques
train_df = compute_indicators(train_df)
test_df = compute_indicators(test_df)

C:\Users\Liam\AppData\Local\Temp\ipykernel_13192\2041586868.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["rsi"] = compute_rsi(train_df["close"], window=14)
C:\Users\Liam\AppData\Local\Temp\ipykernel_13192\2041586868.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["macd"], train_df["macd_signal"] = compute_macd(train_df["close"])
C:\Users\Liam\AppData\Local\Temp\ipykernel_13192\2041586868.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

Recherche des paramètres optimaux de la stratégies

In [12]:
import numpy as np
import pandas as pd
import itertools
ema_short = 5
ema_long = 30
# Fonction pour appliquer la stratégie avec les nouvelles conditions
def apply_strategy(train_df, ema_short, ema_long):
    train_df['ema_short'] = train_df['close'].ewm(span=ema_short).mean()
    train_df['ema_long'] = train_df['close'].ewm(span=ema_long).mean()
    
    train_df['signal'] = 0
    train_df.loc[train_df['ema_short'] > train_df['ema_long'], 'signal'] = 1  # Achat
    train_df.loc[train_df['ema_short'] < train_df['ema_long'], 'signal'] = -1  # Vente
    
    # Conditions supplémentaires
    train_df.loc[(train_df['signal'] == 1) & (train_df['rsi'] > 70), 'signal'] = 0
    train_df.loc[(train_df['signal'] == 1) & (train_df['macd'] < train_df['macd_signal']), 'signal'] = 0
    train_df.loc[(train_df['signal'] == 1) & (train_df['volatility_20d'] > train_df['volatility_20d'].quantile(0.9)), 'signal'] = 0
    train_df.loc[(train_df['signal'] == 1) & (train_df['roc_5'] < 0), 'signal'] = 0
    
    return train_df

# Fonction de backtest
def backtest_strategy(train_df, initial_capital=5000):
    capital = initial_capital
    position = 0
    
    for i in range(len(train_df)):
        signal = train_df['signal'].iloc[i]
        close_price = train_df['close'].iloc[i]
        
        if signal == 1 and position == 0 and capital >= close_price:
            position = 1
            capital -= close_price
        elif signal == -1 and position == 1:
            capital += position * close_price
            position = 0
    
    return capital  # Retourne le capital final

# Optimisation des moyennes mobiles
ema_short_range = range(5, 50, 5)
ema_long_range = range(50, 200, 10)
best_params = None
best_performance = -np.inf

for ema_short, ema_long in itertools.product(ema_short_range, ema_long_range):
    if ema_short >= ema_long:
        continue  # La MA rapide doit être inférieure à la MA lente
    
    train_df = apply_strategy(df.copy(), ema_short, ema_long)
    final_capital = backtest_strategy(train_df)
    
    if final_capital > best_performance:
        best_performance = final_capital
        best_params = (ema_short, ema_long)

print(f'Paramètres optimaux: ema_short={best_params[0]}, ema_long={best_params[1]}')
print(f'Capital final: {best_performance} €')

Paramètres optimaux: ema_short=45, ema_long=190
Capital final: 8058.249999999993 €


Export des df's 

In [ ]:
# Exporter le DataFrame en CSV avec le nom du fichier ajouté au chemin
train_df.to_csv(chemin + "train_df.csv", index=False)
test_df.to_csv(chemin + "test_df.csv", index=False)

Fichier exporté avec succès : C:\Users\Liam\Desktop\Projet crypto\Data\df_export.csvdf_export.csv
